Installing libraries
Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

Importing libraries

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

Search for Target protein
Target search for coronavirus

In [ ]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('aromatase')
targets = pd.DataFrame.from_dict(target_query)
targets

Select and retrieve bioactivity data for SARS coronavirus 3C-like proteinase (fifth entry)
We will assign the fifth entry (which corresponds to the target protein, coronavirus 3C-like proteinase) to the selected_target variable

In [ ]:
selected_target = targets.target_chembl_id[4]
selected_target

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)
df.head(3)

In [ ]:
df.standard_type.unique()

Finally we will save the resulting bioactivity data to a CSV file bioactivity_data.csv.

In [ ]:
df.to_csv('bioactivity_data.csv', index=False)

Copying files to Google Drive
Firstly, we need to mount the Google Drive into Colab so that we can have access to our Google adrive from within Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Next, we create a data folder in our Colab Notebooks folder on Google Drive.

In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data2"

In [ ]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

Let's see the CSV files that we have so far.

In [ ]:
! ls

Taking a glimpse of the bioactivity_data.csv file that we've just created.

In [ ]:
! head bioactivity_data.csv

Handling missing data
If any compounds has missing value for the standard_value column then drop it

In [ ]:
df2 = df[df.standard_value.notna()]
df2

Apparently, for this dataset there is no missing data. But we can use the above code cell for bioactivity data of other target protein.

Data pre-processing of the bioactivity data
Labeling compounds as either being active, inactive or intermediate
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be active while those greater than 10,000 nM will be considered to be inactive. As for those values in between 1,000 and 10,000 nM will be referred to as intermediate.

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

Iterate the molecule_chembl_id to a list

In [ ]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

Iterate canonical_smiles to a list

In [ ]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

Iterate standard_value to a list

In [ ]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

Combine the 4 lists into a dataframe

In [ ]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [ ]:
df3

Alternative method

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2[selection]
df3

In [ ]:
pd.concat([df3,pd.Series(bioactivity_class)], axis=1)

Saves dataframe to CSV file

In [ ]:
df3.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [ ]:
! ls -l

Let's copy to the Google Drive

In [ ]:

! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"